In [15]:
import csv
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [7]:
path = '../data/hw1/covid.train.csv'
test_path = '../data/hw1/covid.test.csv'
feats = list(range(40)) + sorted([75, 57, 42, 60, 78, 43, 61, 79, 40, 58, 76, 41, 59, 77])
data = pd.read_csv('../data/hw1/covid.train.csv')
x = data[data.columns[1:94]]
y = data[data.columns[-1]]

In [15]:
bestfeatures = SelectKBest(score_func=f_regression, k=5)
fit = bestfeatures.fit(x, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)

featureScores = pd.concat([dfcolumns, dfscores], axis=1)
featureScores.columns = ['Specs', 'Score']  #naming the dataframe columns
print(featureScores.nlargest(15, 'Score'))  #print 15 best features

NameError: name 'SelectKBest' is not defined

In [6]:
with open(path, 'r') as fp:
    data2 = list(csv.reader(fp))
    data2 = np.array(data2[1:])[:, 1:].astype(float)
data2 = data2[:, feats]

In [8]:
# 数据预处理 & 筛选特征
def data_prep(path, mode='train'):
    with open(path, 'r') as fp:
        data = list(csv.reader(fp))
        data = np.array(data[1:])[:, 1:].astype(float)

    feats = list(range(40)) + sorted([75, 57, 42, 60, 78, 43, 61, 79, 40, 58, 76, 41, 59, 77])
    if mode == 'test':
        data = data[:, feats]
    else:
        target = data[:, -1]
        data = data[:, feats]

    data[:, 40:] = (data[:, 40:] - np.mean(data[:, 40:], axis=0)) / np.std(data[:, 40:], axis=0)
    if mode == 'test':
        return data
    else:
        return data, target

In [10]:
train_x, train_y = data_prep(path, 'train')
train_x

array([[ 1.        ,  0.        ,  0.        , ..., -0.3176832 ,
        -0.39147861, -0.35256812],
       [ 1.        ,  0.        ,  0.        , ..., -0.13449624,
        -0.4252708 , -0.41037418],
       [ 1.        ,  0.        ,  0.        , ..., -0.14020198,
        -0.4039971 , -0.41919541],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -1.06108356,
        -0.61645386, -0.57179892],
       [ 0.        ,  0.        ,  0.        , ..., -0.97093074,
        -0.64099242, -0.58785131],
       [ 0.        ,  0.        ,  0.        , ..., -1.06543432,
        -0.67737001, -0.6287395 ]])

In [21]:
preds = []
for i in range(10):
    B = torch.rand(1, 1)
    preds.append(B.item())
preds

[0.6477536559104919,
 0.9471188187599182,
 0.07161778211593628,
 0.8423126339912415,
 0.45321375131607056,
 0.6633135676383972,
 0.16640186309814453,
 0.10819387435913086,
 0.5734816193580627,
 0.025683343410491943]

In [26]:
torch.backends.cudnn.deterministic = True